## Load necessary libraries

Instructions for installing the necessary libraries are provided in [Readme.md]() file provided on the project page.

List of libraries which may need to be installed explicitly.


*  tensorflow
*  sklearn
*  scipy
*  pandas
*  numpy
*  matplotlib



In [1]:
from google.colab import drive
drive.mount('/gdrive')
import pathlib

import os

print(os.getcwd())
%cd ..
%cd gdrive
%cd My Drive
%cd MAJOR PROJECT
#data_root = pathlib.Path("/gdrive/My Drive/MAJOR PROJECT")

#%cd ./gdrive/My Drive/MAJOR PROJECT
print(os.getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/content
/
/gdrive
/gdrive/My Drive
/gdrive/My Drive/MAJOR PROJECT
/gdrive/My Drive/MAJOR PROJECT


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import layers

import os
import time

from matplotlib import pyplot as plt
from IPython import display

from scipy.io import loadmat

import numpy as np

import pandas as pd

import sklearn.preprocessing


##Function for loading mri data

The code for handling fMRI data is used as provided in [repository](https://github.com/WeizmannVision/ssfmri2im) for the [paper](https://arxiv.org/abs/1907.02431) (From voxels to pixels and back: Self-supervision in natural-image reconstruction from fMRI).

The fMRI data is normalized by default using standard scaling method.

In [3]:
class kamitani_data_handler():
    """Generate batches for FMRI prediction
    frames_back - how many video frames to take before FMRI frame
    frames_forward - how many video frames to take after FMRI frame
    """

    def __init__(self, matlab_file ,test_img_csv = os.getcwd()+'/imageURL_test.csv',train_img_csv = os.getcwd()+'/imageURL_training.csv',voxel_spacing =3,log = 0 ):
        mat = loadmat(matlab_file)
        self.data = mat['dataSet'][:,3:]
        self.sample_meta = mat['dataSet'][:,:3]
        meta = mat['metaData']
        # print(meta)
        # print(len(self.data[0]))
        #print(len(self.sample_meta))
        self.meta_keys = list(l[0] for l in meta[0][0][0][0])
        self.meta_desc = list(l[0] for l in meta[0][0][1][0])
        self.voxel_meta = np.nan_to_num(meta[0][0][2][:,3:])
        test_img_df = pd.read_csv(test_img_csv, header=None)
        train_img_df =pd.read_csv(train_img_csv, header=None)
        self.test_img_id = test_img_df[1].values
        self.train_img_id = train_img_df[3].values
        # print(len(self.train_img_id))
        self.sample_type = {'train':1 , 'test':2 , 'test_imagine' : 3}
        self.voxel_spacing = voxel_spacing
        img_ids = self.get_meta_field('Label')
        # print(len(img_ids))
        type = self.get_meta_field('DataType')
        train = (type == self.sample_type['train'])
        test = (type == self.sample_type['test'])
        imag = (type == self.sample_type['test_imagine'])

        self.img_ids_train = img_ids[train]
        self.img_ids_test = img_ids[test]
        self.img_ids_imag = img_ids[imag]
        self.log = log

    def get_meta_field(self,field = 'DataType'):
        index = self.meta_keys.index(field)
        if(index <3): # 3 first keys are sample meta
            return self.sample_meta[:,index]
        else:
            return self.voxel_meta[index]


    def print_meta_desc(self):
        print(self.meta_desc)
    def get_data(self,normalize =2 ,roi = 'ROI_VC',imag_data = 0,test_run_list = None):   # normalize 0-no, 1- per run , 2- train/test seperatly
        type = self.get_meta_field('DataType')

        train = (type == self.sample_type['train'])
        test = (type == self.sample_type['test'])
        test_imag = (type == self.sample_type['test_imagine'])
        test_all  = np.logical_or(test,test_imag)
        

        roi_select = self.get_meta_field(roi).astype(bool)
        data = self.data[:,roi_select]

        # if(self.log ==1):
        #     data = np.log(1+np.abs(data))*np.sign(data)


        if(normalize==1):

            run = self.get_meta_field('Run').astype('int')-1
            num_runs = np.max(run)+1
            data_norm = np.zeros(data.shape)

            for r in range(num_runs):
                data_norm[r==run] = sklearn.preprocessing.scale(data[r==run])
            train_data = data_norm[train]
            test_data  = data_norm[test]
            test_all = data_norm[test_all]
            test_imag = data_norm[test_imag]

        else:

            train_data = data[train]
            index = list(range(len(self.img_ids_train)))


            index.sort(key=self.img_ids_train.__getitem__)

            # list1 = [self.img_ids_train[i] for i in index]
            train_data = [train_data[i] for i in index]
            # print(len(train_data[0]))
            # print(list1)
            # print(list2)
            # sortedtest = self.img_ids_test[:50]
            # index = list(range(len(sortedtest)))
            # index.sort(key=sortedtest.__getitem__)
            # # list1 = [sortedtest[i] for i in index]
            # # print(list1)
            # test_data  =  data[test][:50]
            # test_data = [test_data[i] for i in index]
            sortedtest = self.img_ids_test[:]
            index = list(range(len(sortedtest)))
            index.sort(key=sortedtest.__getitem__)
            list1 = [sortedtest[i] for i in index]
            # print(list1)
            test_data  =  data[test][:]
            test_data = [test_data[i] for i in index]
            # print(test_data)
            L=[]
            for i in range(50):
              tp= np.mean(test_data[i*35:(i+1)*35],axis=0)
              L.append(tp)
            test_data=L
            # print(test_data)
            if(normalize==2):
                train_data = sklearn.preprocessing.scale(train_data)
                test_data = sklearn.preprocessing.scale(test_data)
                


        return train_data, test_data


##Loading fMRI data

List of subjects: [ '/Subject1.mat' , '/Subject2.mat' , '/Subject3.mat' , '/Subject4.mat' , '/Subject5.mat' ]

List of ROIs : [ 'ROI_VC' , 'ROI_HVC' , 'ROI_LVC' , 'ROI_V1' , 'ROI_V2' , 'ROI_V3' , 'ROI_V4' , 'ROI_LOC' , 'ROI_FFA' , 'ROI_PPA' ]

In [104]:
subject = "/Subject3.mat"
ROI = 'ROI_VC'

handler = kamitani_data_handler(matlab_file=os.getcwd()+subject)
Y_train, Y_test = handler.get_data(roi=ROI)
IN_SHAPE= len(Y_train[0])

handler=None



##Functions to load images

Following augumentations are applied to all the images.

Resize : ( 128 * 128 )

Normalize: -1 to 1

In [5]:
IMG_SIZE = 128          # Dimension of resized image


def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  input_image = tf.cast(image, tf.float32)
  
  return input_image

def resize(input_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  

  return input_image

def normalize(input_image):
  input_image = (input_image / 127.5) - 1
  

  return input_image


def load_image_train(image_file):
  input_image= load(image_file)
  input_image= resize(input_image,IMG_SIZE, IMG_SIZE)
  input_image= normalize(input_image)

  return input_image

##Loading images

In [6]:
trainimg = tf.data.Dataset.list_files(os.getcwd()+'/TRAINS/*.JPEG',shuffle=False)

testimg = tf.data.Dataset.list_files(os.getcwd()+'/img_test/*.JPEG',shuffle=False)


trainimg = trainimg.map(load_image_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

testimg = testimg.map(load_image_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)


##Loading image features

We use features from the last Convolution layer of [VGG19](https://arxiv.org/abs/1409.1556) for training image generator as well as fMRI decoder.

Code for extracting and saving features is provided in [vgg19features.py]()

In [7]:
import h5py


h5f = h5py.File('trainvgg_data.h5','r')
IM = h5f['train'][:]
h5f.close()
h5f = h5py.File('testvgg_data.h5','r')
IMT = h5f['test'][:]
h5f.close()



##Defining Neural Network Decoder

Dimension of input : fMRI_shape

Number of hidden layers: 2

Dimension of output layer : 8192

We use GaussianDropout as first layer for improving generalization. GaussianDropout applies both, Dropout and Gaussian Noise on the fMRI inputs.

We also use 'SELU' activation in the second hidden layers for adding non-linearity to the network.

Nodes of hidden layers and rate of GaussianDropout can be changed in the arguments while instantiating the decoder. The default values which provide best results are set as :

nodes_1 = 512

nodes_2 = 256

gd_rate = 0.125

In [111]:
def dec(n_1=1024,n_2=512,r=0.12):
  model = tf.keras.Sequential()
  model.add(layers.InputLayer(input_shape=(IN_SHAPE)))

  model.add(layers.GaussianDropout((r)))

  model.add(layers.Dense(n_1,))

  model.add(layers.Dense(n_2,'selu'))

  model.add(layers.Dense(8192,))

  return model


In [153]:
Dec1=dec(256,256,0.15)
# Dec2=dec(256,0.1)
# Dec3=dec(512,)
# Dec4=dec(512,0.1)
Dec1.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])
# Dec2.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])
# Dec3.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])
# Dec4.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])

In [154]:
# tf.random.set_seed(2)
Dec1.fit(Y_train,IM,16,100,verbose=1)

Epoch 1/100
75/75 [==============================] - 0s 3ms/step - loss: 0.3495 - mse: 0.3495 - mae: 0.3380
Epoch 2/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1873 - mse: 0.1873 - mae: 0.2704
Epoch 3/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1566 - mse: 0.1566 - mae: 0.2433
Epoch 4/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1386 - mse: 0.1386 - mae: 0.2258
Epoch 5/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1316 - mse: 0.1316 - mae: 0.2186
Epoch 6/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1247 - mse: 0.1247 - mae: 0.2128
Epoch 7/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1190 - mse: 0.1190 - mae: 0.2089
Epoch 8/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1136 - mse: 0.1136 - mae: 0.2049
Epoch 9/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1110 - mse: 0.1110 - mae: 0.2036
Epoch 10/100
75/75 [========

##Forming input pipeline for image Generator network 

In [155]:

test_feat = Dec1.predict(Y_test)

test_feat= tf.data.Dataset.from_tensor_slices(test_feat)

test_dataset =tf.data.Dataset.zip((test_feat,testimg))

test_dataset = test_dataset.batch(1)




In [ ]:
i=1
for (example_input,target) in test_dataset.take(50):
    
        generate_images(generator, example_input, target,i)
        i+=1

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
ypred1 = Dec1.predict(Y1_test)

# ypred2 = Dec2.predict(Y1_test)

# ypred3 = Dec3.predict(Y1_test)

# ypred4 = Dec4.predict(Y1_test)

# print(mean_squared_error(IMT,ypred))
# print(mean_absolute_error(IMT,ypred))
# print(r2_score(IMT,ypred))


##Loading generator model and defining image generation function

In [114]:
from PIL import Image
generator = tf.keras.models.load_model('GENERATOR8192')
generator.trainable =False

def generate_images(model, test_input, tar,name):
  prediction = model(test_input, training=False)
  plt.figure(figsize=(2,2))

  display_list = [prediction[0]]
  # display_list = [ tar[0], prediction[0]]
  # title = ['Ground Truth', 'Predicted Image']
  plt.imshow(display_list[0] * 0.5 + 0.5)
  # for i in range(1):
  #   plt.subplot(1, 2, i+1)
  #   plt.title(title[i])
  #   # getting the pixel values between [0, 1] to plot it.
  #   plt.imshow(display_list[i] * 0.5 + 0.5)
  plt.axis('off')
  # plt.show()
  plt.savefig(os.getcwd()+'/gen_direct/'+str(name)+'.png')

##generating images from decoder predictions

In [ ]:
# generator = tf.keras.models.load_model('GENERATOR')
# generator.trainable =False
# from PIL import Image

def generate_images(num,model, test_input,avg, tar):
  prediction = model(test_input, training=False)
  plt.figure(figsize=(5,5))
  pred = model(avg, training=False)
  # tf.keras.preprocessing.image.save_img( os.getcwd()+'/gen_direct/'+str(num)+'.png', prediction, data_format=None, file_format=None, scale=True)
  display_list = [ tar[0], prediction[0],pred[0]]
  title = ['Ground Truth', 'Predicted Image','avg']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  # for i in range(1):
  #   plt.subplot(1, 1, i+1)
  #   # plt.title(title[i_1])
  #   # getting the pixel values between [0, 1] to plot it.
  #   plt.imshow(display_list[i+2] * 0.5 + 0.5)
  #   plt.axis('off')
  plt.show()
  # plt.savefig(os.getcwd()+'/gen_direct/'+str(num)+'.png')